In [45]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel
from sklearn.cluster import DBSCAN
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import r2_score
from sklearn.linear_model import Ridge
from sklearn import preprocessing
import xgboost as xgb

In [46]:
# Step 0: Load the proper csvs

xtrain = pd.read_csv("X_train.csv")
xtrain = xtrain.iloc[:, 1:]
xtest = pd.read_csv("X_test.csv")
xtest = xtest.iloc[:, 1:]
ytrain = pd.read_csv("Y_train.csv")

In [47]:
# Step 1: Treat missing values
# Treat missing values as column medians. Important (apparently) to use the medians from the training set in the test set

xtrain = xtrain.fillna(xtrain.median())
xtest = xtest.fillna(xtrain.median())

In [48]:
# Step 2: Scale the training and test data

scaler = preprocessing.StandardScaler()

xtrain_scaled = scaler.fit_transform(xtrain)
xtrain = pd.DataFrame(xtrain_scaled, columns = xtrain.columns)
xtest_scaled = scaler.fit_transform(xtest)
xtest = pd.DataFrame(xtest_scaled, columns = xtest.columns)

In [49]:
# Step 3: Remove unnecessary id column from ytrain that just ***** things up

ytrain1 = ytrain.loc[:, "y"]
ytrain2 = pd.DataFrame(data = ytrain1.values, columns= ['y'])
ytrain = ytrain2

In [50]:
# Outlier detection with local outlier factor
from sklearn.neighbors import LocalOutlierFactor

clf = LocalOutlierFactor(n_neighbors=700, contamination=0.1)
outliers = clf.fit_predict(xtrain)

# Remove outliers from xtrain and ytrain


outliers = outliers == 1
print(np.count_nonzero(outliers))
xtrain = xtrain[outliers]
ytrain = ytrain[outliers]

1090


In [51]:
#Feature selection

# option 1  
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_regression
k_best = 500
xtrain = SelectKBest(mutual_info_regression, k=k_best).fit_transform(xtrain, ytrain)

'''
# option 2
sel = SelectFromModel(RandomForestClassifier(n_estimators = 200))
sel.fit(xtrain, ytrain)
selected_feat = xtrain.columns[(sel.get_support())]
xtrain = xtrain.loc[:,selected_feat]
xtest = xtest.loc[:,selected_feat]
'''


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


'\n# option 2\nsel = SelectFromModel(RandomForestClassifier(n_estimators = 200))\nsel.fit(xtrain, ytrain)\nselected_feat = xtrain.columns[(sel.get_support())]\nxtrain = xtrain.loc[:,selected_feat]\nxtest = xtest.loc[:,selected_feat]\n'

In [52]:
pd.DataFrame(xtrain).head()

,0,1,2,3,4,5,6,7,8,9,...,490,491,492,493,494,495,496,497,498,499
0,2.001809,-0.652740,-1.881312,0.917417,-0.388651,1.845923,-1.112274,0.275672,1.405990,-0.720986,...,-0.002819,-0.381793,-0.897460,1.480634,-0.008390,0.249939,0.312998,-0.945093,0.925679,0.149058
1,0.555346,-0.460321,2.108910,1.206184,-0.070530,0.464995,1.754254,1.231450,-0.700342,-0.342019,...,-1.609892,-0.622497,-0.014061,-0.112038,0.454378,1.251713,-1.442390,-0.207605,0.265215,1.292065
2,-0.110118,0.526563,-0.286194,0.523945,0.225598,-0.843648,-0.569341,0.210957,-0.870321,1.267598,...,-0.193833,0.810331,0.472682,-1.637788,1.091439,-1.411038,0.415870,-1.072406,-0.031837,-2.122182
3,0.572714,1.801777,1.736020,1.073601,-1.513891,-0.542565,-0.000429,-0.611077,-1.751880,0.916094,...,1.718758,-0.626668,-0.634381,1.645562,0.691200,-0.192713,-0.335371,-1.156034,0.304935,-1.064809
4,-0.319577,0.148305,-1.195911,-0.288568,0.364310,-1.150942,-0.000429,0.629295,1.409429,1.033598,...,1.368602,-0.519776,-0.330124,-1.303248,-0.762968,-0.617725,-1.020345,0.277899,0.431110,0.585331


In [70]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.linear_model import LinearRegression
from sklearn import linear_model

folds = 10

cv_mean_scores = []
cv_stds = []
estimators = np.array([100, 200, 300, 400])
alphas = np.array([0.01, 0.03, 0.1, 0.3, 1, 3])
mutual_features = np.array([50, 100, 200, 300, 400])
max_depth = np.array([5, 10, 20, 30])



for n in estimators:
    for best in mutual_features:
        for a in alphas: 
            for depth in max_depth: 
                
                xtrain = SelectKBest(mutual_info_regression, k=best).fit_transform(xtrain, ytrain)
                r1 = RandomForestRegressor(n_estimators= n, random_state=42, max_depth= depth, bootstrap = True)
                r2 = linear_model.Lasso(alpha= a)
                r3 = xgb.XGBRegressor(objective  = "reg:squarederror", random_state = 42, n_estimators=200)
                er = VotingRegressor([('rf', r1), ('lasso', r2), ('xgb', r3)])
                scores = cross_val_score(estimator = er, X = xtrain,
                                   y = ytrain, scoring = 'r2', cv = folds)
                cv_mean_scores.append(np.mean(scores))   
                cv_stds.append(np.std(scores))

    
print(cv_mean_scores)
print(cv_stds)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\voting.py:43: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X, y)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\voting.py:43: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X, y)


KeyboardInterrupt: 